In [1]:
const dfd = require("danfojs-node");
const tf = dfd.tensorflow
// const tf = require('@tensorflow/tfjs');

In [2]:
config = {
  rowStart: 7228,
  rowEnd: 7288,
  timesteps: 2,
  maxBatchSize: 1,
  layerSize: 1,
  unitSize: 1,
  dropout: 0,
  learningRate: 0.1,
  maxEpochs: 50,
  timeCol: 'tanggal',
  feature: ['rr']
}

// usecols = [config.timeCol].concat(config.feature)

function trainTestSplit(dataset, timeStep = 1) {
  dataX = []
  dataY = []

  // for (let i = 0; i < dataset.values.length - timeStep - 1; i++) {
  for (let i = 0; i < dataset.values.length - timeStep; i++) {
    dataX.push(dataset.iloc({ rows: [`${i}:${i + timeStep}`] }).values)
    dataY.push(dataset.iloc({ rows: [i + timeStep] }).values[0])
  }

  return [dataX, dataY]
}

{
  rowStart: 7228,
  rowEnd: 7288,
  timesteps: 2,
  maxBatchSize: 1,
  layerSize: 1,
  unitSize: 1,
  dropout: 0,
  learningRate: 0.1,
  maxEpochs: 50,
  timeCol: 'tanggal',
  feature: [ 'rr' ]
}

In [15]:
function sigmod(x) {
  return 1 / (1 + Math.exp(-x))
}

function tanh(x) {
  return Math.tanh(x)
}

In [16]:
sigmod(1)

0.7310585786300049

In [17]:
tanh(1)

0.7615941559557649

In [3]:
$$async$$ = true;
(async function () {
  dataset = await dfd.readCSV("../Data/1985-2021.csv")
  dataset = dataset.iloc({rows:[`${config.rowStart}:${config.rowEnd}`]})
  datelist = dataset.loc({columns: [config.timeCol]})

  $$done$$(dataset.shape)
})()

[ 60, 11 ]

In [4]:
featureset = dataset.loc({columns: config.feature})

scaller = new dfd.MinMaxScaler()
scaller.fit(featureset)

featuresetScaled = scaller.transform(featureset)

trainSize = featuresetScaled.values.length * 0.9
trainset = featuresetScaled.iloc({rows: [`${0}:${trainSize}`]})
testset =  featuresetScaled.iloc({rows: [`${trainSize}:${featuresetScaled.values.length}`]})

trainset = trainTestSplit(trainset, timeStep=config.timesteps)
testset = trainTestSplit(testset, timeStep=config.timesteps)

XTrain = tf.tensor(trainset[0])
yTrain = tf.tensor(trainset[1])

XTest = tf.tensor(testset[0])
yTest = tf.tensor(testset[1])

XTrain = tf.reshape(XTrain, [XTrain.shape[0], config.feature.length, XTrain.shape[1]])
XTest = tf.reshape(XTest, [XTest.shape[0], config.feature.length, XTest.shape[1]])

Tensor {
  kept: false,
  isDisposedInternal: false,
  shape: [ 4, 1, 2 ],
  dtype: 'float32',
  size: 8,
  strides: [ 2, 2 ],
  dataId: {},
  id: 18,
  rankType: '3',
  scopeId: 6
}

In [5]:
yPredLoss = []
yTrueLoss = []
loss = []

function meanSquaredError(yTrue, yPred) {
  yPredLoss.push(yPred.array())
  yTrueLoss.push(yTrue.array())
  loss.push(tf.losses.meanSquaredError(yTrue, yPred).arraySync())
  return tf.losses.meanSquaredError(yTrue, yPred)
}

model = tf.sequential()

model.add(tf.layers.lstm({
  units: config.unitSize,
  batchInputShape: [config.maxBatchSize, config.feature.length, config.timesteps],
  goBackwards: true,
  dropout: config.dropout,
  weights: [
    tf.tensor([
      [0.5774, 0.5774, 0.5774, 0.5774],
      [0.5774, 0.5774, 0.5774, 0.5774]
    ]),
    tf.tensor([
      [0.5774, 0.5774, 0.5774, 0.5774]
    ]),
    tf.zeros([4])
  ]
}))

// model.layers[0].getWeights()[0].print()

model.compile({
  optimizer: tf.train.sgd(config.learningRate),
  // loss: 'meanSquaredError',
  loss: meanSquaredError,
  runEagerly: true
})

model.summary()

__________________________________________________________________________________________
Layer (type)                Input Shape               Output shape              Param #   
lstm_LSTM1 (LSTM)           [[1,1,2]]                 [1,1]                     16        
Total params: 16
Trainable params: 16
Non-trainable params: 0
__________________________________________________________________________________________


In [6]:
batchEnd = []
epochEnd = []

$$async$$ = true;
(async function () {
  history = await model.fit(XTrain, yTrain, {
    shuffle: false,
    epochs: config.maxEpochs,
    verbose: 1,
    batchSize: config.maxBatchSize,
    callbacks: [
      new tf.CustomCallback({
        // onEpochBegin: null,
        onEpochEnd: (epoch, logs = null) => epochEnd.push(logs.loss),
        // on_batch_begin: null,
        onBatchEnd: (batch, logs = null) => batchEnd.push(logs.loss),
        // onTrainBegin: null,
        // onTrainEnd: null,
      })
    ]
  })
  $$done$$()
})()

In [7]:
$$async$$ = true;
(async function () {
  await Promise.all(yPredLoss).then(values => {
    console.log(values);
  });
  $$done$$()
})()

[
  [ [ 0.003764435416087508 ] ],
  [ [ 0.2075117528438568 ] ],
  [ [ 0.31454363465309143 ] ],
  [ [ 0.24471434950828552 ] ],
  [ [ 0.18366461992263794 ] ],
  [ [ 0.07705558836460114 ] ],
  [ [ 0.06722110509872437 ] ],
  [ [ 0.05115007609128952 ] ],
  [ [ 0.02120816893875599 ] ],
  [ [ 0.045577529817819595 ] ],
  [ [ 0.06173235550522804 ] ],
  [ [ 0.08842892944812775 ] ],
  [ [ 0.10123305767774582 ] ],
  [ [ 0.13989442586898804 ] ],
  [ [ 0.13641346991062164 ] ],
  [ [ 0.06277725100517273 ] ],
  [ [ 0.07123468816280365 ] ],
  [ [ 0.09615403413772583 ] ],
  [ [ 0.11529918015003204 ] ],
  [ [ 0.1393214911222458 ] ],
  [ [ 0.14054806530475616 ] ],
  [ [ 0.08195114135742188 ] ],
  [ [ 0.06584157794713974 ] ],
  [ [ 0.10253032296895981 ] ],
  [ [ 0.07829129695892334 ] ],
  [ [ 0.18813830614089966 ] ],
  [ [ 0.19023026525974274 ] ],
  [ [ 0.11310435831546783 ] ],
  [ [ 0.2378578782081604 ] ],
  [ [ 0.2208230197429657 ] ],
  [ [ 0.20531068742275238 ] ],
  [ [ 0.16387724876403809 ] ],
  [ [ 0.

In [8]:
batchEnd

[
       0.9924852848052979,    0.10129033029079437,   0.09368187189102173,
  0.000057774414017330855,   0.006491821259260178,  0.009643659926950932,
    0.0021717974450439215,   0.002511928789317608,   0.01780383288860321,
    0.0019459737231954932,    0.04647785425186157,  0.004115016665309668,
       0.1149027869105339,   5.172122996555117e-7,  0.004816798958927393,
     0.012651857919991016,   0.014031832106411457,  0.026777978986501694,
     0.032608095556497574,    0.01402103528380394,  0.016962140798568726,
     0.006384733133018017,   0.016909027472138405, 0.0037568577099591494,
       0.3890896737575531,    0.01543080247938633,  0.014172444120049477,
       0.2877036929130554, 0.00012218552001286298,    0.1080184057354927,
     0.030050890520215034,    0.02486666850745678,  0.002926415763795376,
    0.0006631721626035869,   0.005667124874889851,    0.6042335629463196,
     0.058012526482343674,   0.007149661425501108,  0.010607315227389336,
     0.013511737808585167, 0.0001276

In [9]:
epochEnd

[
  0.07985345274209976,  0.0716899037361145,  0.0690271183848381,
  0.06783075630664825,  0.0671655684709549, 0.06673196703195572,
  0.06641414016485214, 0.06616140902042389, 0.06594928354024887,
  0.06576499342918396, 0.06560125201940536, 0.06545361131429672,
  0.06531914323568344, 0.06519577652215958, 0.06508197635412216,
  0.06497654318809509, 0.06487848609685898, 0.06478700786828995,
  0.06470144540071487,  0.0646212100982666, 0.06454578787088394,
  0.06447473913431168, 0.06440768390893936, 0.06434430181980133,
  0.06428425014019012, 0.06422726064920425,  0.0641731321811676,
  0.06412162631750107, 0.06407254189252853, 0.06402570009231567,
  0.06398095935583115, 0.06393817067146301, 0.06389720737934113,
  0.06385792791843414,  0.0638202428817749, 0.06378404796123505,
   0.0637492686510086, 0.06371579319238663, 0.06368356943130493,
  0.06365251541137695, 0.06362255662679672, 0.06359364092350006,
  0.06356573104858398, 0.06353874504566193,  0.0635126605629921,
  0.06348742544651031, 

In [10]:
proyeksi = model.predict(XTest, yTest)

Tensor {
  kept: false,
  isDisposedInternal: false,
  shape: [ 4, 1 ],
  dtype: 'float32',
  size: 4,
  strides: [ 1 ],
  dataId: {},
  id: 666015,
  rankType: '2',
  scopeId: 566981
}

In [11]:
proyeksi.dataSync()

Float32Array(4) [
  0.1997341513633728,
  0.187837153673172,
  0.18746259808540344,
  0.18775293231010437
]